In [4]:
from fastai.text import load_learner
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
from deep_translator import GoogleTranslator
import warnings
from collections import Counter

warnings.filterwarnings('ignore')

In [5]:
sentiment_model = load_learner('model_v2')

In [6]:
str(sentiment_model.predict("आज़तक पर देखिये कितनी स्मार्ट हुई स्मार्ट फेंसिंग Exclusive आज रात 9.30 pm पर @BSF_India")[0])

'negative'

# Extract News Headlines From Finviz

In [17]:
finviz_url = 'https://finviz.com/quote.ashx?t='
ticker = 'ADBE'
news_tables = {}
url = finviz_url + ticker
req = Request(url=url, headers={'user-agent': 'my-app'})
response = urlopen(req)
html = BeautifulSoup(response, 'html')
news_table = html.find(id='news-table')
news_tables[ticker] = news_table

# Overall Sentiment Test

In [18]:
sentiment = []
for ticker, news_table in news_tables.items():
    for row in news_table.findAll('tr'):
        if row.a and row.a.text:
            title = row.a.get_text()
            sentiment.append(str(sentiment_model.predict(GoogleTranslator(source='auto', target='hi').translate(title))[0]))

sentiment_counts = Counter(sentiment)
overall_sentiment = max(sentiment_counts, key=sentiment_counts.get)
overall_sentiment

'negative'

# Today Sentiment Test

In [19]:
parsed_data = []
date = ''

for ticker, news_table in news_tables.items():
    for row in news_table.findAll('tr'):
        if row.a and row.a.text:
            title = row.a.get_text()
            date_data = [item for item in row.td.text.split(' ') if item.strip()]
            if len(date_data) == 2:
                date = date_data[0]
            if date == 'Dec-10-23':
                parsed_data.append([ticker, date, title])
            else:
                break
            
sentiment = []
for text in parsed_data:
    sentiment.append(str(sentiment_model.predict(GoogleTranslator(source='auto', target='hi').translate(text[2]))[0]))

    sentiment_counts = Counter(sentiment)
today_sentiment = max(sentiment_counts, key=sentiment_counts.get)
today_sentiment

'negative'